In [ ]:
import os
import time
import requests
import pandas as pd
import zipfile
import io
import re
import concurrent.futures
from datetime import datetime

def homologar_estado(nombre):
    if not isinstance(nombre, str): return nombre
    n = nombre.lower()
    
    if 'ciudad de méxico' in n or 'ciudad de mexico' in n or 'cdmx' in n or 'distrito federal' in n: return 'Ciudad de México'
    if 'baja california sur' in n: return 'Baja California Sur'
    if 'baja california' in n: return 'Baja California'
    if 'estado de méxico' in n or 'estado de mexico' in n or n.strip() == 'méxico' or n.strip() == 'mexico': return 'México'
    
    if 'coahuila' in n: return 'Coahuila'
    if 'michoacán' in n or 'michoacan' in n: return 'Michoacán'
    if 'veracruz' in n: return 'Veracruz'
    if 'nuevo león' in n or 'nuevo leon' in n: return 'Nuevo León'
    if 'querétaro' in n or 'queretaro' in n: return 'Querétaro'
    if 'san luis' in n: return 'San Luis Potosí'
    if 'yucatán' in n or 'yucatan' in n: return 'Yucatán'
    
    estados = ['Aguascalientes', 'Campeche', 'Colima', 'Chiapas', 'Chihuahua', 
               'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 
               'Morelos', 'Nayarit', 'Oaxaca', 'Puebla', 'Quintana Roo', 
               'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Zacatecas']
    
    for est in estados:
        if est.lower() in n: return est
        
    return nombre

def limpiar_columna_estado(df):
    nombres_validos = ['estado', 'entidad', 'entidad federativa', 'estados', 'entidades']
    for col in df.columns:
        if str(col).lower().strip() in nombres_validos:
            df[col] = df[col].apply(homologar_estado)
    return df

# ==========================================
# 0. CONFIGURACIÓN GLOBAL Y RUTAS
# ==========================================
TOKEN_INEGI = "129ac2e3-e8a6-72c7-58c1-acced5a601bd"

# Detección robusta de directorios
try:
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
    if os.path.basename(SCRIPT_DIR) == 'scripts':
        PROJECT_ROOT = os.path.dirname(SCRIPT_DIR)
    else:
        PROJECT_ROOT = SCRIPT_DIR
except NameError:
    PROJECT_ROOT = os.getcwd()
    if os.path.basename(PROJECT_ROOT) == 'scripts':
        PROJECT_ROOT = os.path.dirname(PROJECT_ROOT)

RAW_DIR = os.path.join(PROJECT_ROOT, "data", "raw")
INTERMEDIATE_DIR = os.path.join(PROJECT_ROOT, "data", "intermediate")

os.makedirs(INTERMEDIATE_DIR, exist_ok=True)

print(f"📍 Raíz del Proyecto: {PROJECT_ROOT}")
print(f"📂 Datos Crudos: {RAW_DIR}")
print(f"📂 Datos Procesados: {INTERMEDIATE_DIR}")
print("-" * 80)

# ==========================================
# MÓDULO 1: PIB (API)
# ==========================================
def procesar_pib():
    print("⏳ [PIB] Iniciando extracción exhaustiva...")
    indicadores = {
        "746097": "Total Nacional",
        "746196": "Actividades Primarias",
        "746229": "Agricultura, cría y explotación de animales, aprovechamiento forestal, pesca y caza",
        "746262": "Agricultura",
        "746295": "Cría y explotación de animales",
        "746328": "Pesca, caza y captura",
        "746361": "Aprovechamiento forestal",
        "746394": "Actividades Secundarias",
        "746427": "Minería",
        "746460": "Minería petrolera",
        "746493": "Minería no petrolera",
        "746526": "Generación, transmisión y distribución de energía eléctrica, agua y gas",
        "746559": "Construcción",
        "746592": "Industrias manufactureras",
        "746625": "Industria alimentaria",
        "746658": "Bebidas y tabaco",
        "746691": "Insumos, acabados y productos textiles",
        "746724": "Prendas de vestir y productos de cuero y piel",
        "746757": "Industria de la madera",
        "746790": "Industria del papel",
        "746823": "Productos derivados del petróleo y carbón, química, plástico y hule",
        "746856": "Productos a base de minerales no metálicos",
        "746889": "Metálicas básicas y productos metálicos",
        "746922": "Maquinaria y equipo, computación, electrónicos y accesorios",
        "746955": "Muebles, colchones y persianas",
        "746988": "Otras industrias manufactureras",
        "747021": "Actividades Terciarias",
        "747054": "Comercio al por mayor",
        "747087": "Comercio al por menor",
        "747120": "Transportes, correos y almacenamiento",
        "747153": "Información en medios masivos",
        "747186": "Servicios financieros y de seguros",
        "747219": "Servicios inmobiliarios y de alquiler de bienes",
        "747252": "Servicios profesionales, científicos y técnicos",
        "747285": "Corporativos",
        "747318": "Servicios de apoyo a los negocios y manejo de residuos",
        "747351": "Servicios educativos",
        "747384": "Servicios de salud y de asistencia social",
        "747417": "Servicios de esparcimiento culturales y deportivos",
        "747450": "Servicios de alojamiento temporal y de preparación de alimentos y bebidas",
        "747483": "Otros servicios excepto actividades gubernamentales",
        "747516": "Actividades legislativas, gubernamentales"
    }
    
    resultados = []
    
    for ind_clave, ind_nombre in indicadores.items():
        for i in range(0, 33): 
            clave_estado = f"{i:02d}"
            url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ind_clave}/es/{clave_estado}/false/BIE-BISE/2.0/{TOKEN_INEGI}?type=json"
            
            exito = False
            for intento in range(3):
                try:
                    r = requests.get(url, timeout=10)
                    if r.status_code == 200:
                        data = r.json()
                        if 'Series' in data and data['Series']:
                            serie = data['Series'][0].get('OBSERVATIONS', [])
                            serie_sorted = sorted(serie, key=lambda x: x.get('TIME_PERIOD', ''))
                            if len(serie_sorted) >= 2:
                                obs_list = serie_sorted[-2:]
                            else:
                                obs_list = serie_sorted
                                
                            for obs in obs_list:
                                resultados.append({
                                    'Indicador': ind_nombre,
                                    'Clave_Indicador': ind_clave,
                                    'Estado_ID': obs.get('COBER_GEO', clave_estado),
                                    'Periodo': int(obs.get('TIME_PERIOD')),
                                    'Valor': float(obs.get('OBS_VALUE', 0))
                                })
                        exito = True
                        break
                    else:
                        time.sleep(1)
                except Exception:
                    time.sleep(1)
            time.sleep(0.05)
            
    if resultados:
        df = pd.DataFrame(resultados)
        outfile = os.path.join(INTERMEDIATE_DIR, "pib_entidad.csv")
        df.to_csv(outfile, index=False)
        return f"✅ [PIB] Completado ({len(df)} registros)."
    return "⚠️ [PIB] No se obtuvieron datos."

# ==========================================
# MÓDULO 2: EXPORTACIONES (API)
# ==========================================
def procesar_exportaciones():
    print("⏳ [Exportaciones] Iniciando extracción detallada...")
    indicadores = {
        "629659": "Total",
        "696790": "Agricultura",
        "696791": "Cría y explotación de animales",
        "697788": "Pesca, caza y captura",
        "629660": "Extracción de petróleo y gas",
        "629661": "Minería de minerales metálicos y no metálicos",
        "629662": "Industria alimentaria",
        "629663": "Industria de las bebidas y del tabaco",
        "629664": "Fabricación de insumos textiles y acabado de textiles",
        "629665": "Fabricación de productos textiles, excepto prendas de vestir",
        "629666": "Fabricación de prendas de vestir",
        "629667": "Curtido y acabado de cuero y piel, y fabricación de productos de cuero",
        "629668": "Industria de la madera",
        "629669": "Industria del papel",
        "629670": "Impresión e industrias conexas",
        "629671": "Fabricación de productos derivados del petróleo y del carbón",
        "629672": "Industria química",
        "629673": "Industria del plástico y del hule",
        "629674": "Fabricación de productos a base de minerales no metálicos",
        "629675": "Industrias metálicas básicas",
        "629676": "Fabricación de productos metálicos",
        "629677": "Fabricación de maquinaria y equipo",
        "629678": "Fabricación de equipo de computación, comunicación, medición y otros equipos, componentes y accesorios electrónicos",
        "629679": "Fabricación de accesorios, aparatos eléctricos y equipo de generación de energía eléctrica",
        "629680": "Fabricación de equipo de transporte",
        "629681": "Fabricación de muebles, colchones y persianas",
        "629682": "Otras industrias manufactureras",
        "629683": "No especificado"
    }
    
    resultados = []
    
    for ind_clave, ind_nombre in indicadores.items():
        for i in range(1, 33):
            clave_estado = f"{i:02d}"
            url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ind_clave}/es/{clave_estado}/false/BIE-BISE/2.0/{TOKEN_INEGI}?type=json"
            
            exito = False
            for intento in range(3):
                try:
                    r = requests.get(url, timeout=10)
                    if r.status_code == 200:
                        data = r.json()
                        if 'Series' in data and data['Series']:
                            serie = data['Series'][0].get('OBSERVATIONS', [])
                            serie_sorted = sorted(serie, key=lambda x: x.get('TIME_PERIOD', ''))
                            if serie_sorted:
                                max_year_str = serie_sorted[-1]['TIME_PERIOD'][:4]
                                try:
                                    max_year = int(max_year_str)
                                    min_year_target = max_year - 1
                                    for obs in serie_sorted:
                                        anio_obs = int(obs['TIME_PERIOD'][:4])
                                        if anio_obs >= min_year_target:
                                            resultados.append({
                                                'Sector': ind_nombre,
                                                'Clave_Indicador': ind_clave,
                                                'Estado_ID': clave_estado,
                                                'Periodo': obs.get('TIME_PERIOD'),
                                                'Valor': float(obs.get('OBS_VALUE', 0))
                                            })
                                except: pass
                        exito = True
                        break
                    else:
                        time.sleep(1)
                except Exception:
                    time.sleep(1)
            time.sleep(0.05)
            
    if resultados:
        df = pd.DataFrame(resultados)
        outfile = os.path.join(INTERMEDIATE_DIR, "exportaciones_entidad.csv")
        df.to_csv(outfile, index=False)
        return f"✅ [Exportaciones] Completado ({len(df)} registros)."
    return "⚠️ [Exportaciones] No se obtuvieron datos."

# ==========================================
# MÓDULO 3: POBLACIÓN (API)
# ==========================================
def procesar_poblacion_api():
    print("⏳ [Población] Iniciando extracción de pirámide completa...")
    indicadores = {
        # Hombres
        "1002000059": "0 a 4 años (Hombres)",
        "1002000089": "5 a 9 años (Hombres)",
        "1002000062": "10 a 14 años (Hombres)",
        "1002000068": "15 a 19 años (Hombres)",
        "1002000071": "20 a 24 años (Hombres)",
        "1002000074": "25 a 29 años (Hombres)",
        "1002000077": "30 a 34 años (Hombres)",
        "1002000080": "35 a 39 años (Hombres)",
        "1002000083": "40 a 44 años (Hombres)",
        "1002000086": "45 a 49 años (Hombres)",
        "1002000092": "50 a 54 años (Hombres)",
        "1002000095": "55 a 59 años (Hombres)",
        "1002000098": "60 a 64 años (Hombres)",
        "1002000101": "65 a 69 años (Hombres)",
        "1002000104": "70 a 74 años (Hombres)",
        "1002000107": "75 a 79 años (Hombres)",
        "1002000110": "80 a 84 años (Hombres)",
        "1002000113": "85 a 89 años (Hombres)",
        "1002000116": "90 a 94 años (Hombres)",
        "1002000119": "95 a 99 años (Hombres)",
        "1002000065": "100 años y más (Hombres)",

        # Mujeres
        "1002000060": "0 a 4 años (Mujeres)",
        "1002000090": "5 a 9 años (Mujeres)",
        "1002000063": "10 a 14 años (Mujeres)",
        "1002000069": "15 a 19 años (Mujeres)",
        "1002000072": "20 a 24 años (Mujeres)",
        "1002000075": "25 a 29 años (Mujeres)",
        "1002000078": "30 a 34 años (Mujeres)",
        "1002000081": "35 a 39 años (Mujeres)",
        "1002000084": "40 a 44 años (Mujeres)",
        "1002000087": "45 a 49 años (Mujeres)",
        "1002000093": "50 a 54 años (Mujeres)",
        "1002000096": "55 a 59 años (Mujeres)",
        "1002000099": "60 a 64 años (Mujeres)",
        "1002000102": "65 a 69 años (Mujeres)",
        "1002000105": "70 a 74 años (Mujeres)",
        "1002000108": "75 a 79 años (Mujeres)",
        "1002000111": "80 a 84 años (Mujeres)",
        "1002000114": "85 a 89 años (Mujeres)",
        "1002000117": "90 a 94 años (Mujeres)",
        "1002000120": "95 a 99 años (Mujeres)",
        "1002000066": "100 años y más (Mujeres)"
}
    
    resultados = []
    
    for ind_clave, desc in indicadores.items():
        for i in range(0, 33):
            url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ind_clave}/es/{i:02d}/true/BISE/2.0/{TOKEN_INEGI}?type=json"
            
            exito = False
            for intento in range(3):
                try:
                    r = requests.get(url, timeout=5)
                    if r.status_code == 200:
                        data = r.json()
                        if 'Series' in data and data['Series']:
                            obs = data['Series'][0]['OBSERVATIONS'][0]
                            resultados.append({
                                'Indicador': desc,
                                'Clave_Indicador': ind_clave,
                                'Estado_ID': f"{i:02d}",
                                'Periodo': obs.get('TIME_PERIOD'),
                                'Valor': float(obs.get('OBS_VALUE', 0))
                            })
                        exito = True
                        break
                    else:
                        time.sleep(0.5)
                except Exception:
                    time.sleep(0.5)
            time.sleep(0.02)
            
    if resultados:
        df = pd.DataFrame(resultados)
        outfile = os.path.join(INTERMEDIATE_DIR, "poblacion_edad.csv")
        df.to_csv(outfile, index=False)
        return f"✅ [Población] Completado ({len(df)} registros)."
    return "⚠️ [Población] No se obtuvieron datos."

# ==========================================
# MÓDULO 4: ENOE (Descarga + PEA Corregida)
# ==========================================
def procesar_enoe_auto():
    print("⏳ [ENOE] Iniciando descarga y procesamiento...")
    
    base_url = "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/tabulados/"
    anio_actual = datetime.now().year
    url_final, anio_found, trim_found = None, None, None
    
    encontrado = False
    for a in [anio_actual, anio_actual-1]:
        if encontrado: break
        for t in ["trim4", "trim3", "trim2", "trim1"]:
            test_url = f"{base_url}enoe_indicadores_estrategicos_{a}_{t}_xls.zip"
            try:
                r = requests.head(test_url, timeout=5)
                content_type = r.headers.get('Content-Type', '').lower()
                if r.status_code == 200 and ('zip' in content_type or 'octet-stream' in content_type):
                    url_final = test_url; anio_found = a; trim_found = t; encontrado = True
                    break
            except: pass
            
    if not url_final: return "❌ [ENOE] URL no encontrada."

    try:
        print(f"   📥 Descargando: {url_final}")
        r = requests.get(url_final)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        
        archivos = [f for f in z.namelist() if ("Entidades/" in f or "Nacional/" in f) and (f.endswith('.xlsx') or f.endswith('.xls'))]
        
        datos = []
        
        config_enoe = {
            "Poblacion Total": ["", "Población total"],
            "PEA": ["", "Población económicamente activa (PEA)"],
            "Desocupada": ["Población económicamente activa", "Desocupada"],
            "Edad Promedio PEA": ["Edad de la población económicamente activa", "Promedio"],
            "Sector Primario": ["3.2 Sector de actividad", "Primario"],
            "Sector Secundario": ["3.2 Sector de actividad", "Secundario"],
            "Sector Terciario": ["3.2 Sector de actividad", "Terciario"],
            "No especificado": ["3.2 Sector de actividad", "No especificado"],
            "Educacion Sup": ["Nivel de instrucción", "Medio superior y superior"],
            "Informalidad TIL1": ["", "Tasa de informalidad laboral 1 (TIL1)"]
        }

        for arch in archivos:
            if "Nacional" in arch:
                estado = "Nacional"
            else:
                estado = arch.split("Entidad_")[-1].replace(".xlsx", "").replace(".xls", "").replace("_", " ").title()
                
            with z.open(arch) as f:
                df = pd.read_excel(f, header=None)
            
            registro = {'Estado': estado, 'Anio': anio_found, 'Trimestre': trim_found}
            
            col_val = 4 
            for idx, row in df.iterrows():
                txt = " ".join([str(x).lower() for x in row[:5]])
                if "población total" in txt:
                    for c in range(4, min(15, len(row))):
                        try:
                            if float(str(row[c]).replace(",","").replace(" ", "")) > 1000: 
                                col_val = c; break
                        except: continue
                    break
            
            context = {i:"" for i in range(5)}
            sticky = {1:""}
            
            for idx, row in df.iterrows():
                cols = [str(row[i]).strip() if i < len(row) and pd.notna(row[i]) else "" for i in range(5)]
                indent = -1
                txt_row = ""
                
                for i, txt in enumerate(cols):
                    if txt:
                        indent = i; txt_row = txt; context[i] = txt
                        for j in range(i+1, 5): 
                            context[j] = ""; 
                            if j in sticky: sticky[j]=""
                        break
                
                if not txt_row: continue
                
                if indent == 1 and re.match(r'^(\d+\.?\d*)\s', txt_row): sticky[1] = txt_row
                
                path = [context[0]]
                if indent == 1 and not re.match(r'^(\d+\.?\d*)\s', txt_row) and sticky[1]:
                    path.append(sticky[1]); path.append(txt_row)
                else: path.append(context[1])
                path += [context[k] for k in range(2,5)]
                
                path_str = " | ".join([p.lower() for p in path if p])
                
                for kpi, (padre, target) in config_enoe.items():
                    if target.lower() in txt_row.lower() and padre.lower() in path_str:
                        try: 
                            val_str = str(row[col_val]).replace(",","").replace(" ", "")
                            registro[kpi] = float(val_str)
                        except: pass
            
            datos.append(registro)

        df_out = pd.DataFrame(datos)
        df_out = limpiar_columna_estado(df_out)
        outfile = os.path.join(INTERMEDIATE_DIR, "enoe_indicadores.csv")
        df_out.to_csv(outfile, index=False)
        return f"✅ [ENOE] Completado ({anio_found}-{trim_found})."
        
    except Exception as e: return f"❌ [ENOE] Error: {e}"

# ==========================================
# MÓDULO 5: EDUCACIÓN (Local)
# ==========================================
def procesar_educacion():
    print("⏳ [Educación] Procesando anuario (Generando Top 3 separados)...")
    
    archivos = [f for f in os.listdir(RAW_DIR) if f.startswith("base_anuario_") and f.endswith(".xlsx")]
    if not archivos:
        return "⚠️ [Educación] Falta archivo base_anuario_####-####.xlsx"
    
    archivo_anuario = archivos[0]
    fpath = os.path.join(RAW_DIR, archivo_anuario)
    
    match = re.search(r'base_anuario_(\d{4}-\d{4})\.xlsx', archivo_anuario)
    ciclo_val = match.group(1) if match else "¿?"
    
    try:
        df = pd.read_excel(fpath, sheet_name="Base de datos")
        
        # Limpieza de columnas numéricas
        cols_num = ['Matrícula Total', 'Egresados Total']
        for c in cols_num: 
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)
        
        mapa_niveles = {
            'TÉCNICO SUPERIOR': 'Técnico Superior', 'LICENCIATURA EN EDUCACIÓN NORMAL': 'Licenciatura',
            'LICENCIATURA UNIVERSITARIA Y TECNOLÓGICA': 'Licenciatura', 'ESPECIALIDAD': 'Licenciatura',
            'MAESTRÍA': 'Maestría', 'DOCTORADO': 'Doctorado'
        }
        df['Nivel_Agrupado'] = df['NIVEL'].str.upper().str.strip().map(mapa_niveles)
        df = limpiar_columna_estado(df)
        
        # --- 1. TOTALES (INTACTO) ---
        df_totales = df.groupby(['ENTIDAD', 'Nivel_Agrupado'])[cols_num].sum().reset_index()
        df_totales['Ciclo'] = ciclo_val
        df_totales.to_csv(os.path.join(INTERMEDIATE_DIR, "educacion_totales.csv"), index=False)
        
        df_campos = df.groupby(['ENTIDAD', 'Nivel_Agrupado', 'CAMPO AMPLIO'])[cols_num].sum().reset_index()
        
        df_nivels = df_campos.groupby(['ENTIDAD', 'Nivel_Agrupado'])[cols_num].sum().reset_index().rename(columns={
            'Matrícula Total': 'Total_Mat', 
            'Egresados Total': 'Total_Egr'
        })
        df_campos = df_campos.merge(df_nivels, on=['ENTIDAD', 'Nivel_Agrupado'])
        
        df_campos['Participacion_Matricula'] = df_campos.apply(lambda x: (x['Matrícula Total']/x['Total_Mat']*100) if x['Total_Mat']>0 else 0, axis=1)
        df_campos['Participacion_Egresados'] = df_campos.apply(lambda x: (x['Egresados Total']/x['Total_Egr']*100) if x['Total_Egr']>0 else 0, axis=1)
        
        top_mat = df_campos.sort_values(['ENTIDAD', 'Nivel_Agrupado', 'Matrícula Total'], ascending=[True, True, False])
        top_mat = top_mat.groupby(['ENTIDAD', 'Nivel_Agrupado']).head(3).copy()
        top_mat['Ciclo'] = ciclo_val
        
        cols_mat = ['ENTIDAD', 'Nivel_Agrupado', 'CAMPO AMPLIO', 'Matrícula Total', 'Participacion_Matricula', 'Ciclo']
        top_mat[cols_mat].to_csv(os.path.join(INTERMEDIATE_DIR, "educacion_top3_matricula.csv"), index=False)
        
        top_egr = df_campos.sort_values(['ENTIDAD', 'Nivel_Agrupado', 'Egresados Total'], ascending=[True, True, False])
        top_egr = top_egr.groupby(['ENTIDAD', 'Nivel_Agrupado']).head(3).copy()
        top_egr['Ciclo'] = ciclo_val
        
        cols_egr = ['ENTIDAD', 'Nivel_Agrupado', 'CAMPO AMPLIO', 'Egresados Total', 'Participacion_Egresados', 'Ciclo']
        top_egr[cols_egr].to_csv(os.path.join(INTERMEDIATE_DIR, "educacion_top3_egresados.csv"), index=False)
        
        return f"✅ [Educación] Completado (Totales + 2 Archivos Top3 - Ciclo {ciclo_val})."
        
    except Exception as e: return f"❌ [Educación] Error: {e}"
# ==========================================
# MÓDULO 6: IED (Local) - GRUPOS POR SECTOR Y SIN %
# ==========================================
def procesar_ied():
    print("⏳ [IED] Procesando datos complejos (Totales 3 Dígitos y Detalle)...")
    fpath = os.path.join(RAW_DIR, "2025_3T_Flujosporentidadfederativa_orig__11_A1.xlsx")
    
    if not os.path.exists(fpath): 
        return f"⚠️ [IED] Falta {fpath}"
    
    try:
        # 1. MAPEO DE COLUMNAS
        df_head = pd.read_excel(fpath, sheet_name='Actividad económica_SCIAN 2023', header=None, nrows=10)
        
        header_idx = None
        for idx, row in df_head.iterrows():
            if "Entidad Federativa" in str(row[0]):
                header_idx = idx
                break
        
        if header_idx is None: return "❌ [IED] Sin encabezados."
        
        years_row = df_head.iloc[header_idx].tolist()
        quarters_row = df_head.iloc[header_idx + 1].tolist()
        
        col_map = {}
        current_year = None
        for i in range(1, len(years_row)):
            if pd.notna(years_row[i]):
                try:
                    y = int(float(years_row[i]))
                    if y > 2000: current_year = y
                except: pass
            if current_year and pd.notna(quarters_row[i]):
                try:
                    q = int(float(quarters_row[i]))
                    if 1 <= q <= 4: col_map[i] = (current_year, q)
                except: pass
        
        if not col_map: return "❌ [IED] Error mapeo columnas."
        
        last_period = max(col_map.values()) # (Año, Trim)
        prev_period = (last_period[0]-1, last_period[1])
        
        idx_act = [k for k, v in col_map.items() if v == last_period][0]
        idx_prev = [k for k, v in col_map.items() if v == prev_period]
        idx_prev = idx_prev[0] if idx_prev else None
        
        # 2. EXTRACCIÓN (Solo 3 dígitos)
        df_data = pd.read_excel(fpath, sheet_name='Actividad económica_SCIAN 2023', header=header_idx+2)
        
        def clean(x):
            if pd.isna(x): return 0.0
            s = str(x).strip().replace(',','')
            try: return float(s)
            except: return 0.0
            
        def clasificar(cod):
            if cod.startswith('1'): return 'Primaria'
            if cod.startswith(('2','3')): return 'Secundaria'
            return 'Terciaria'
            
        filas = []
        estado_act = None
        
        for i, row in df_data.iterrows():
            c = str(row.iloc[0]).strip()
            if not c or c == 'nan': continue
            
            match = re.match(r'^(\d{2,6}|31-33)\s+(.*)', c)
            if match:
                cod, desc = match.groups()
                # Filtro Estricto: Solo 3 dígitos
                if len(cod) == 3 and cod != '31-33':
                    val_act = clean(row.iloc[idx_act])
                    val_prev = clean(row.iloc[idx_prev]) if idx_prev else 0.0
                    
                    filas.append({
                        'Estado': estado_act,
                        'Codigo': cod,
                        'Actividad': desc,
                        'Sector': clasificar(cod), 
                        'Inversion': val_act, 
                        'Inversion_Anterior': val_prev
                    })
            elif not c[0].isdigit() and "total" not in c.lower() and "nota" not in c.lower():
                estado_act = c
        
        df_clean = pd.DataFrame(filas)
        if not df_clean.empty:
            df_clean = limpiar_columna_estado(df_clean)

        if not df_clean.empty:
            df_totales = df_clean.groupby(['Estado', 'Sector'])[['Inversion', 'Inversion_Anterior']].sum().reset_index()
            
            df_totales['Anio'] = last_period[0]
            df_totales['Trimestre'] = last_period[1]
            
            df_totales.to_csv(os.path.join(INTERMEDIATE_DIR, "ied_totales.csv"), index=False)
            
            tops = []
            for est in df_clean['Estado'].unique():
                d_est = df_clean[df_clean['Estado'] == est]
                for sec in ['Primaria', 'Secundaria', 'Terciaria']:
                    d_sec = d_est[d_est['Sector'] == sec]
                    top3 = d_sec.sort_values('Inversion', ascending=False).head(3).copy()
                    tops.append(top3)
            
            if tops:
                df_tops = pd.concat(tops)
                
                df_tops['Anio'] = last_period[0]
                df_tops['Trimestre'] = last_period[1]
                
                df_tops.to_csv(os.path.join(INTERMEDIATE_DIR, "ied_top3_sectores.csv"), index=False)
                return f"✅ [IED] Completado (Periodo {last_period})."
        
        return "⚠️ [IED] Sin datos extraídos."
        
    except Exception as e: return f"❌ [IED] Error: {e}"

# ==========================================
# MÓDULO 7: SAIC (Local)
# ==========================================
def procesar_saic():
    print("⏳ [SAIC] Procesando censo...")
    fpath = os.path.join(RAW_DIR, "SAIC.xlsx")
    if not os.path.exists(fpath): return f"⚠️ [SAIC] Falta {fpath}"
    
    try:
        df = pd.read_excel(fpath, header=4)
        df = df.iloc[:, [0, 1, 3, 4]]
        df.columns = ['Anio_Censal', 'Entidad', 'Personal_Ocupado', 'Produccion_Bruta']
        
        df = df.dropna(how='all')
        df = df[~df['Anio_Censal'].astype(str).str.lower().str.startswith('nota')]
        df = df.dropna(subset=['Entidad'])
        
        def clean_entidad(val):
            return re.sub(r'^\d+\s+', '', str(val).strip())

        def clean_numeric(val):
            if pd.isna(val): return 0.0
            if isinstance(val, str):
                val = val.replace(',', '').strip()
                if val == '' or val == '-': return 0.0
            return float(val)

        df['Entidad'] = df['Entidad'].apply(clean_entidad)
        df = limpiar_columna_estado(df)
        df['Personal_Ocupado'] = df['Personal_Ocupado'].apply(clean_numeric)
        df['Produccion_Bruta'] = df['Produccion_Bruta'].apply(clean_numeric)

        df['Indicador_Productividad'] = df.apply(
            lambda row: (row['Produccion_Bruta'] / row['Personal_Ocupado']) * 1000
            if row['Personal_Ocupado'] != 0 else 0.0, 
            axis=1
        )
        
        cols_finales = ['Anio_Censal', 'Entidad', 'Personal_Ocupado', 'Produccion_Bruta', 'Indicador_Productividad']
        df = df[cols_finales]
        
        df.to_csv(os.path.join(INTERMEDIATE_DIR, "saic_productividad.csv"), index=False)
        return "✅ [SAIC] Completado."
        
    except Exception as e: return f"❌ [SAIC] Error: {e}"

# ==========================================
# MÓDULO 8: IMCO (Local)
# ==========================================
def procesar_imco():
    print("⏳ [IMCO] Procesando competitividad...")
    f_gen = os.path.join(RAW_DIR, "imco_general.csv")
    f_des = os.path.join(RAW_DIR, "imco_desagregado.csv")
    if not os.path.exists(f_gen) or not os.path.exists(f_des): return "⚠️ [IMCO] Faltan archivos."
    
    try:
        # General
        try: df_g = pd.read_csv(f_gen, encoding='utf-8')
        except: df_g = pd.read_csv(f_gen, encoding='latin-1')
        cols_map = {c: 'Año' if 'AÃ±o' in c else 'Cambio' if 'posiciÃ³n' in c else c for c in df_g.columns}
        df_g.rename(columns=cols_map, inplace=True)
        df_g = df_g[df_g['Año'] == df_g['Año'].max()]
        df_g = limpiar_columna_estado(df_g)
        df_g.to_csv(os.path.join(INTERMEDIATE_DIR, "imco_general_final.csv"), index=False)
        
        # Desagregado
        try: df_d = pd.read_csv(f_des, encoding='utf-8')
        except: df_d = pd.read_csv(f_des, encoding='latin-1')
        cols_map_d = {c: 'Subíndice' if 'SubÃ­ndice' in c else c for c in df_d.columns}
        df_d.rename(columns=cols_map_d, inplace=True)
        df_d = limpiar_columna_estado(df_d)
        
        fechas = sorted(df_d['Date'].unique(), reverse=True)[:2]
        df_d = df_d[df_d['Date'].isin(fechas)].copy()
        df_d['Rank'] = df_d.groupby(['Date', 'Indicador'])['Value'].rank(ascending=False, method='min')
        
        if len(fechas) >= 2:
            df_curr = df_d[df_d['Date'] == fechas[0]].copy()
            df_prev = df_d[df_d['Date'] == fechas[1]][['Entidad', 'Indicador', 'Rank']].rename(columns={'Rank': 'Rank_Prev'})
            df_fin = df_curr.merge(df_prev, on=['Entidad', 'Indicador'], how='left')
            df_fin['Cambio_Posicion'] = df_fin['Rank_Prev'] - df_fin['Rank']
            df_fin['Cambio_Posicion'] = df_fin['Cambio_Posicion'].fillna(0)
        else:
            df_fin = df_d; df_fin['Cambio_Posicion'] = 0
            
        df_fin.to_csv(os.path.join(INTERMEDIATE_DIR, "imco_desagregado_final.csv"), index=False)
        return "✅ [IMCO] Completado."
    except Exception as e: return f"❌ [IMCO] Error: {e}"

# ==========================================
# ORQUESTADOR
# ==========================================
def main():
    inicio = time.time()
    print("\n🚀 INICIANDO ETL ESTATAL UNIFICADO 🚀\n")
    
    tareas = [
        procesar_pib, procesar_exportaciones, procesar_poblacion_api, 
        procesar_enoe_auto, procesar_educacion, procesar_ied, 
        procesar_saic, procesar_imco
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        futuros = {executor.submit(t): t.__name__ for t in tareas}
        for futuro in concurrent.futures.as_completed(futuros):
            print(f"{futuro.result()}")
                
    print(f"\n✨ PROCESO TERMINADO EN {time.time()-inicio:.2f} SEGUNDOS ✨")
    print(f"📂 Archivos en: {INTERMEDIATE_DIR}")

if __name__ == "__main__":
    main()

📍 Raíz del Proyecto: c:\Users\Edward\Desktop\Bancomext\Estatales
📂 Datos Crudos: c:\Users\Edward\Desktop\Bancomext\Estatales\data\raw
📂 Datos Procesados: c:\Users\Edward\Desktop\Bancomext\Estatales\data\intermediate
--------------------------------------------------------------------------------

🚀 INICIANDO ETL ESTATAL UNIFICADO 🚀

⏳ [PIB] Iniciando extracción exhaustiva...
⏳ [Exportaciones] Iniciando extracción detallada...
⏳ [Población] Iniciando extracción de pirámide completa...
⏳ [ENOE] Iniciando descarga y procesamiento...
⏳ [Educación] Procesando anuario (Generando Top 3 separados)...
⏳ [IED] Procesando datos complejos (Totales 3 Dígitos y Detalle)...
⏳ [SAIC] Procesando censo...
⏳ [IMCO] Procesando competitividad...
✅ [IMCO] Completado.
✅ [SAIC] Completado.
✅ [Educación] Completado (Totales + 2 Archivos Top3 - Ciclo 2024-2025).
   📥 Descargando: https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/tabulados/enoe_indicadores_estrategicos_2025_trim3_xls.zip
✅ [IED] Completa